# DataGrab

Author: Bobby Schulz, schu3119@umn.edu || Last Updated: May 30, 2024

### Function
Grab data from GEMS Sensing database based on specified values of project, node id, and timeframe.

Returns results as a [pickle](https://docs.python.org/3/library/pickle.html), a CSV, or xlsx.

### Reqirements 
1. You must have valid credentials to access the GEMS Sensing database. If you do not already have credentials, email Bryan Runck (runck014@umn.edu) to get access. 
2. You must be connected to eduroam at a UMN campus **OR** be connected to a UMN network via [VPN](https://it.umn.edu/services-technologies/virtual-private-network-vpn) 

Acnowledgements: Based on work by Bryan Runck regarding data access 

#### Import all needed packages

In [3]:
# import pkg_resources
from sqlalchemy import create_engine # engine to access database
import pandas as pd # used to store and manipulate the data
import os # needed for basic IO
import json # needed for parsing of json info
from datetime import datetime # used to handle date time information for parsing
import time # used to keep track of current time for measuring execution time
# import numpy as np # used for basic maths
# import matplotlib.pyplot as plt
# print(dir(create_engine))

### Access credential file

A credential file (`credentials.json`) is required in the same working directory that this notebook is located. The file must be laid out in the defined manor. See example in [NotActualCredentials.json](./NotActualCredentials.json)

In [18]:
# Define the file path; default assumes same directory as this notebook
file_path = "credentials.json"

if os.path.isfile(file_path):
    #If file exists, open the file for reading
    with open(file_path, "r") as file:
        credentials = json.load(file)
    # Set Username and Password
    try:
        username = credentials['db_username']
        password = credentials['db_password']
        host = credentials['host']
        port = credentials['port']
        db = credentials['db']
    except:
        print("ERROR: One or more of the expected values is missing!")
        print("\tCheck your credential file for formatting and presance of db_username, db_password, host, port, and db")
else:
    print("ERROR: No credentials file found!")
    print("\tPlease check location and existance of credentials.json")

print("DONE - Credential info imported from file")

DONE - Credential info imported from file


#### Create postgresql engine 
Create engine for accessing database using the credential info previously imported

In [19]:
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}')

### Define node names, projects, and time frames
In the following cell the user must define the node ids of the desired nodes, along with the projects where we expect to find them, and the time frames that data should be returned.

**Example input:** 
```
node_ids = "('INT_GEMS_2', 'GEMS_6')"
projects = {'eroc', 'stellenbosch')"
timeStart = '2023-08-16 12:30:00'
timeStop = '2024-03-01 16:00:00;
```

**Note: Program will search for ALL node ids in ALL projects listed. As long as the node id exists in one of those projects, it will be found**

In [27]:
node_ids = "('MET1_MAES')"
projects = {'eroc'}
timeStart = "2023-08-16"
timeStop = "2024-03-01"

### Execute data grab based on user input

#### Please be patient! This may take some time

In [31]:
#Multi-Node, Multi-Project Query - GEMS
# get data by passing a query string and database engine to pandas read_sql

gemsData = pd.DataFrame() #Make empty dataframe to hold all the results
print("Begin SQL Query - Please be patient, this may take some time\n")
print('{:>20s}{:^10s}{:<10s}'.format('Project', '', 'Time [s]'))
print('-'*40)
for proj in projects:
    query = "SELECT * FROM " +  proj + ".data WHERE display_name in " + node_ids + " AND \"time\" > '" + timeStart + "' AND \"time\" < '" + timeStop + "'" #Concatonate values to make call to specific project and from your custom list of nodes
# query = "SELECT * FROM eroc.data LIMIT 1000"
    print('{:>20s}{:^10s}'.format(proj, ''), end =" ") #Print project name and blank space, but don't print newline so we can print time on same line when done
    start = time.time() #Keep track of how long execute takes
    gemsData = pd.concat([gemsData, pd.read_sql(query, engine)]) #Concatonate data together so you get one dataframe at end
    end = time.time()
    print('{:<5.0f}'.format((end-start)))
print("\nDone")

nodeVals = node_ids[1:-1].split(',')
nodeVals = [s.strip("'") for s in nodeVals] #Extract list of node names from string of node names

missingNodes = set(nodeVals).difference(set(gemsData['display_name'].unique()))
if len(missingNodes) > 0: #Only print out if there any missing nodes
    print("\n\nMissing Nodes:") 
    for node in missingNodes: #Print out list of any missing nodes
        print("\t", node)
        
print(gemsData.head(10)) #Print out sample of data

print("Done! Query completed")

Begin SQL Query - Please be patient, this may take some time

             Project          Time [s]  
----------------------------------------
                eroc           111  

Done
                 time      value                   node_id           measure  \
0 2023-08-16 00:10:54  70.906067  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
1 2023-08-16 00:28:48  71.275330  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
2 2023-08-16 00:44:02  72.145081  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
3 2023-08-16 00:59:17  72.673035  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
4 2023-08-16 01:14:32  73.196411  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
5 2023-08-16 01:32:21  73.210144  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
6 2023-08-16 01:47:42  73.022461  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
7 2023-08-16 02:18:26  72.926331  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
8 2023-08-16 02:03:04  73.025513  e00fce68055fc955f7d19b4a  SHTC3.0.HUMIDITY   
9 2023-08-16 

### Save Data
Proceed below to save the data in one of various formats

- CSV
    - The most generalizeable form of data storage, but not as efficient as Pickle for reading from, etc. Good for universal access.
- XLSX
    - Excel proprietary format. Helpful for those who are forced to work within Excel, but not for much else. 
- Pickle
    - File type used by Python to store large amounts of data. This is helpful if you want to load this data into another Python enviroment and analyize it. 

| Format | Ease of Use (Novice) | Ease of Use (Analyist) | Modularity | File Size | Write Time | Read Time |
| ------ | -------------------- | ---------------------- | ---------- | --------- | ---------- | --------- |
| CSV | Good | Good | Best | Worst | Good | Good |
| XLSX | Best | Poor | Poor | Best | WORST | WORST |
| Pickle | Worst | Best | Worst | Ok | Best | Best |

Example:

269k records (one node, about 6 months)

| Format | Write Time | Read Time | File Size | 
| ------ | ---------- | --------- | --------- |
| CSV | 1.18s | 240ms | 21.6 MB |
| XLSX | 21s | 17s | 7.2 MB |
| Pickle | 210ms | 110ms | 19.7 MB | 


In [40]:
if not os.path.isdir("GEMS_Data"): #If there is not already a data folder, make one
    os.mkdir("GEMS_Data") #Make a folder to keep data in for saving to prevent clutter
timeStr = str(datetime.now().replace(microsecond=0).isoformat()) #Grab ISO 8601 format YYYY-mm-ddTHH:MM:SS
timeStr = timeStr.replace('-', '') #Strip out hypens for file naming
timeStr = timeStr.replace(':', '') #Strip out colons for file naming

# Comment out the following to ommit any save options
gemsData.to_csv('GEMS_Data/Data_' + timeStr + '.csv', index=None) #CSV
gemsData.to_excel('GEMS_Data/Data_' + timeStr + '.xlsx', index=None) #XLSX
gemsData.to_pickle('GEMS_Data/Data_' + timeStr + '.pkl') #PICKLE

1717092561.3715267
1717092562.5549514
1717092562.766386
